# le get et post ( Recup de secret dans coffre fort "keyvolts")  
# PARQUET

In [8]:
# 1. Import des bibliothèques nécessaires
from azure.identity import ClientSecretCredential, DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from azure.storage.filedatalake import DataLakeServiceClient
import pandas as pd
import os
import io

from datasets import load_dataset
import tempfile


import requests
import pandas as pd
from azure.core.exceptions import AzureError
from concurrent.futures import ThreadPoolExecutor, as_completed


from dotenv import load_dotenv




In [9]:

# 2. Chargement des variables d'environnement
load_dotenv()

# 2. Configuration des informations d'authentification
# Configuration Key Vault
key_vault_url = os.getenv('KEY_VAULT_URL')
secret_name = os.getenv('WRITE_SECRET_NAME')

# Configuration Azure AD
tenant_id = os.getenv('TENANT_ID')
client_id = os.getenv('WRITE_CLIENT_ID')

# Configuration Storage
storage_account_name = os.getenv('STORAGE_ACCOUNT_NAME')
container_name = os.getenv('CONTAINER_NAME')


In [10]:

# 3. Récupération du secret depuis Key Vault
def get_secret_from_keyvault():
    try:
        # Utilisation de DefaultAzureCredential pour l'authentification
        credential = DefaultAzureCredential()
        
        # Création du client Key Vault
        secret_client = SecretClient(vault_url=key_vault_url, credential=credential)
        
        # Récupération du secret
        secret = secret_client.get_secret(secret_name)
        return secret.value
        
    except Exception as e:
        print(f"Erreur lors de la récupération du secret: {str(e)}")
        return None

In [11]:
def create_credentials():
    client_secret = get_secret_from_keyvault()
    if client_secret:
        return ClientSecretCredential(
            tenant_id=tenant_id,
            client_id=client_id,
            client_secret=client_secret
        )
    return None

In [12]:

# 5. Fonction pour uploader le fichier vers Data Lake
def upload_to_datalake(local_file_path, destination_path):
    try:
        # Obtention des credentials
        credential = create_credentials()
        if not credential:
            raise Exception("Impossible d'obtenir les credentials")

        # URL du service
        account_url = f"https://{storage_account_name}.dfs.core.windows.net"
        
        # Création du client service
        service_client = DataLakeServiceClient(account_url, credential=credential)
        
        # Obtention du client système de fichiers
        file_system_client = service_client.get_file_system_client(file_system=container_name)
        
        # Création du client fichier
        file_client = file_system_client.create_file(destination_path)
        
        # Lecture et upload du fichier
        with open(local_file_path, 'rb') as local_file:
            file_contents = local_file.read()
            
        file_client.append_data(data=file_contents, offset=0, length=len(file_contents))
        file_client.flush_data(len(file_contents))
            
        print(f"Fichier uploadé avec succès vers: {destination_path}")
        
    except Exception as e:
        print(f"Une erreur est survenue: {str(e)}")

# Fonction pour télécharger et uploader les fichiers Parquet depuis Hugging Face


# Niveau 1 avec 1 fichier 




In [13]:



# def process_single_parquet_file():
#     try:
#         # URL du fichier
#         file_url = "https://huggingface.co/datasets/Marqo/amazon-products-eval/resolve/main/data/data-00000-of-00105.parquet"
        
#         print("📥 Début du transfert du fichier vers Data Lake...")
        
#         # Configuration de la requête
#         headers = {
#             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
#         }
        
#         # Initialisation de la requête en streaming
#         response = requests.get(file_url, headers=headers, stream=True)
#         response.raise_for_status()
        
#         # Obtention de la taille totale du fichier
#         total_size = int(response.headers.get('content-length', 0))
        
#         # Configuration du client Data Lake
#         credential = create_credentials()
#         if not credential:
#             raise Exception("Impossible d'obtenir les credentials")

#         account_url = f"https://{storage_account_name}.dfs.core.windows.net"
#         service_client = DataLakeServiceClient(account_url, credential=credential)
#         file_system_client = service_client.get_file_system_client(container_name)
        
#         # Création du fichier dans le Data Lake
#         destination_path = "huggingface_data/data-00000-of-00105.parquet"
#         file_client = file_system_client.create_file(destination_path)
        
#         # Transfert direct des données
#         current_position = 0
#         for chunk in response.iter_content(chunk_size=4*1024*1024):  # 4MB chunks
#             if chunk:
#                 file_client.append_data(data=chunk, offset=current_position)
#                 current_position += len(chunk)
                
#                 # Affichage de la progression
#                 if total_size:
#                     progress = (current_position / total_size) * 100
#                     print(f"\rProgression du transfert: {progress:.2f}% ({current_position/(1024*1024):.2f} Mo / {total_size/(1024*1024):.2f} Mo)", end='')

#         # Finalisation du fichier
#         file_client.flush_data(current_position)
#         print("\n✅ Transfert terminé avec succès!")
        
#         print(f"Taille totale transférée: {current_position/(1024*1024):.2f} Mo")

#     except Exception as e:
#         print(f"\n❌ Une erreur est survenue: {str(e)}")

# def verify_uploaded_files():
#     try:
#         credential = create_credentials()
#         if not credential:
#             raise Exception("Impossible d'obtenir les credentials")

#         account_url = f"https://{storage_account_name}.dfs.core.windows.net"
#         service_client = DataLakeServiceClient(account_url, credential=credential)
#         file_system_client = service_client.get_file_system_client(container_name)
        
#         print("\n📋 Fichiers uploadés dans le Data Lake:")
#         print("-" * 50)
        
#         paths = file_system_client.get_paths(path="huggingface_data")
#         for path in paths:
#             size_mb = path.content_length / (1024 * 1024)
#             print(f"📄 {path.name} ({size_mb:.2f} Mo)")
            
#     except Exception as e:
#         print(f"❌ Erreur lors de la vérification: {str(e)}")

# if __name__ == "__main__":
#     print("🚀 Démarrage du traitement...")
#     process_single_parquet_file()
#     verify_uploaded_files()


# NIVEAU 2 -- Mode Batch --

In [ ]:


def transfer_single_file(file_number, base_url, file_system_client):
    """Fonction pour transférer un seul fichier"""
    try:
        file_name = f"data-{str(file_number).zfill(5)}-of-00105.parquet"
        file_url = f"{base_url}{file_name}"
        destination_path = f"huggingface_data/{file_name}"
        
        print(f"\n📥 Début du transfert du fichier {file_name}...")
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        
        response = requests.get(file_url, headers=headers, stream=True)
        response.raise_for_status()
        
        total_size = int(response.headers.get('content-length', 0))
        
        file_client = file_system_client.create_file(destination_path)
        
        current_position = 0
        for chunk in response.iter_content(chunk_size=4*1024*1024):
            if chunk:
                file_client.append_data(data=chunk, offset=current_position)
                current_position += len(chunk)
                
                if total_size:
                    progress = (current_position / total_size) * 100
                    print(f"\r{file_name}: {progress:.2f}% ({current_position/(1024*1024):.2f} Mo / {total_size/(1024*1024):.2f} Mo)", end='')

        file_client.flush_data(current_position)
        print(f"\n✅ {file_name} transféré avec succès! ({current_position/(1024*1024):.2f} Mo)")
        return True, file_name, current_position

    except Exception as e:
        print(f"\n❌ Erreur lors du transfert de {file_name}: {str(e)}")
        return False, file_name, 0

def process_multiple_files(start_file=0, num_files=5, max_workers=3):
    """Fonction principale pour traiter plusieurs fichiers en parallèle"""
    try:
        print(f"🚀 Démarrage du transfert batch ({num_files} fichiers)...")
        
        base_url = "https://huggingface.co/datasets/Marqo/amazon-products-eval/resolve/main/data/"
        
        # Configuration du client Data Lake
        credential = create_credentials()
        if not credential:
            raise Exception("Impossible d'obtenir les credentials")

        account_url = f"https://{storage_account_name}.dfs.core.windows.net"
        service_client = DataLakeServiceClient(account_url, credential=credential)
        file_system_client = service_client.get_file_system_client(container_name)
        
        # Traitement parallèle des fichiers
        successful_transfers = 0
        total_size = 0
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_file = {
                executor.submit(
                    transfer_single_file, 
                    file_num, 
                    base_url, 
                    file_system_client
                ): file_num 
                for file_num in range(start_file, start_file + num_files)
            }
            
            for future in as_completed(future_to_file):
                success, file_name, size = future.result()
                if success:
                    successful_transfers += 1
                    total_size += size

        print(f"\n📊 Résumé du transfert batch:")
        print(f"- Fichiers transférés avec succès: {successful_transfers}/{num_files}")
        print(f"- Taille totale transférée: {total_size/(1024*1024*1024):.2f} Go")

    except Exception as e:
        print(f"❌ Erreur générale: {str(e)}")

def verify_uploaded_files():
    """Fonction pour vérifier les fichiers uploadés"""
    try:
        credential = create_credentials()
        if not credential:
            raise Exception("Impossible d'obtenir les credentials")

        account_url = f"https://{storage_account_name}.dfs.core.windows.net"
        service_client = DataLakeServiceClient(account_url, credential=credential)
        file_system_client = service_client.get_file_system_client(container_name)
        
        print("\n📋 Fichiers dans le Data Lake:")
        print("-" * 50)
        
        total_size = 0
        file_count = 0
        
        paths = file_system_client.get_paths(path="huggingface_data")
        for path in paths:
            size_mb = path.content_length / (1024 * 1024)
            print(f"📄 {path.name} ({size_mb:.2f} Mo)")
            total_size += path.content_length
            file_count += 1
            
        print("\n📊 Résumé:")
        print(f"- Nombre total de fichiers: {file_count}")
        print(f"- Taille totale: {total_size/(1024*1024*1024):.2f} Go")
            
    except Exception as e:
        print(f"❌ Erreur lors de la vérification: {str(e)}")

if __name__ == "__main__":
    # Configuration du batch
    START_FILE = 0       # Numéro du premier fichier
    NUM_FILES = 5        # Nombre de fichiers à traiter
    MAX_WORKERS = 3      # Nombre de téléchargements simultanés
    
    print("🎯 Configuration du batch:")
    print(f"- Fichiers {START_FILE} à {START_FILE + NUM_FILES - 1}")
    print(f"- {MAX_WORKERS} téléchargements simultanés")
    
    # Exécution du transfert batch
    process_multiple_files(START_FILE, NUM_FILES, MAX_WORKERS)
    
    # Vérification finale
    verify_uploaded_files()


🎯 Configuration du batch:
- Fichiers 0 à 4
- 3 téléchargements simultanés
🚀 Démarrage du transfert batch (5 fichiers)...

📥 Début du transfert du fichier data-00000-of-00105.parquet...
📥 Début du transfert du fichier data-00001-of-00105.parquet...


📥 Début du transfert du fichier data-00002-of-00105.parquet...
data-00000-of-00105.parquet: 100.00% (439.64 Mo / 439.64 Mo)
✅ data-00000-of-00105.parquet transféré avec succès! (439.64 Mo)

📥 Début du transfert du fichier data-00003-of-00105.parquet...
data-00002-of-00105.parquet: 100.00% (447.48 Mo / 447.48 Mo)
✅ data-00002-of-00105.parquet transféré avec succès! (447.48 Mo)

📥 Début du transfert du fichier data-00004-of-00105.parquet...
data-00001-of-00105.parquet: 100.00% (448.80 Mo / 448.80 Mo)
✅ data-00001-of-00105.parquet transféré avec succès! (448.80 Mo)
data-00003-of-00105.parquet: 36.05% (164.00 Mo / 454.94 Mo)